# Oleobrás

A Oleobrás é uma empresa de mineração. O objetivo do projeto é descobrir a melhor região para construção de novos poços de petróleo. 
Foi disponibilizado dados sobre amostras de petróleo de três regiões. Os parâmetros de cada poço de petróleo na região já são conhecidos. O modelo foi construído para escolher a região com probabilidade maior de margem de lucro mais alta.

## Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

## Preparando os dados

* id — identificador unívoco de poço de petróleo
* f0, f1, f2 — três características de pontos (seu significado específico não é importante, mas as próprias características são significativas)
* product — volume de reservas no poço de petróleo (milhares de barris).

In [2]:
# Lendo arquivos csvs
region_1 = pd.read_csv('geo_data_0.csv')
region_2 = pd.read_csv('geo_data_1.csv')
region_3 = pd.read_csv('geo_data_2.csv')

In [3]:
region_1.head(10)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


In [4]:
region_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
region_2.head(10)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


In [6]:
region_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
region_3.head(10)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


In [8]:
region_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
print(f'Valores duplicados nos dados da região 1: {region_1.duplicated().sum()}')
print(f'Valores duplicados nos dados da região 2: {region_2.duplicated().sum()}') 
print(f'Valores duplicados nos dados da região 3: {region_3.duplicated().sum()}')

Valores duplicados nos dados da região 1: 0
Valores duplicados nos dados da região 2: 0
Valores duplicados nos dados da região 3: 0


## Treinando modelo para cada região

In [10]:
# Separando dataset em objetivo e características região 1
target_r1 = region_1['product']
features_r1 = region_1[['f0', 'f1', 'f2']]

# Separando dataset em objetivo e características região 2
target_r2 = region_2['product']
features_r2 = region_2[['f0', 'f1', 'f2']]

# Separando dataset em objetivo e características região 3
target_r3 = region_3['product']
features_r3 = region_3[['f0', 'f1', 'f2']]

In [11]:
# Separando dataset em conjunto de validação e treinamento região 1
features_train_r1, features_valid_r1, target_train_r1, target_valid_r1 = train_test_split(
    features_r1, target_r1, test_size=0.25, random_state=12345)

# Separando dataset em conjunto de validação e treinamento região 2
features_train_r2, features_valid_r2, target_train_r2, target_valid_r2 = train_test_split(
    features_r2, target_r2, test_size=0.25, random_state=12345)

# Separando dataset em conjunto de validação e treinamento região 3
features_train_r3, features_valid_r3, target_train_r3, target_valid_r3 = train_test_split(
    features_r3, target_r3, test_size=0.25, random_state=12345)

In [12]:
model = LinearRegression()
model.fit(features_train_r1, target_train_r1)
predictions_valid_r1 = model.predict(features_valid_r1)
rqm_r1 = mean_squared_error(target_valid_r1, predictions_valid_r1)**0.5

print(f'O volume médio predito pelo modelo é {predictions_valid_r1.mean()}')
print(f'O REQM do modelo é {rqm_r1}')

O volume médio predito pelo modelo é 92.59256778438035
O REQM do modelo é 37.5794217150813


In [13]:
model = LinearRegression()
model.fit(features_train_r2, target_train_r2)
predictions_valid_r2 = model.predict(features_valid_r2)
rqm_r2 = mean_squared_error(target_valid_r2, predictions_valid_r2)**0.5

print(f'O volume médio predito pelo modelo é {predictions_valid_r2.mean()}')
print(f'O REQM do modelo é {rqm_r2}')

O volume médio predito pelo modelo é 68.728546895446
O REQM do modelo é 0.893099286775617


In [14]:
model = LinearRegression()
model.fit(features_train_r3, target_train_r3)
predictions_valid_r3 = model.predict(features_valid_r3)
rqm_r3 = mean_squared_error(target_valid_r3, predictions_valid_r3)**0.5

print(f'O volume médio predito pelo modelo é {predictions_valid_r3.mean()}')
print(f'O REQM do modelo é {rqm_r3}')

O volume médio predito pelo modelo é 94.96504596800489
O REQM do modelo é 40.02970873393434


A região que há previsão de maior lucro médio é a região 3,  entretanto possui REQM mais alto também. O REQM mostra a diferença média entre os valores previstos e reais. A região 2 é possui previsões menos lucrativas. Já a região 1 tem previsões relativamente aproximadas a região 3.

## Cálculo de lucro


In [15]:
budget = 100000000
product_price = 4500

In [17]:
# Função para calcular o preço 

def calculate_profit(predictions):
 
    # Selecinando os poços com maiores volumes
    top_wells = sorted(predictions, reverse=True)[:200]
    
    # Somando volume total
    total_product = sum(top_wells)
    
    # Calculando a receita
    revenue = total_product * product_price
    
    # Calculando o lucro
    profit = revenue - budget
   
    return profit


In [18]:
profit_r1 = calculate_profit(predictions_valid_r1)
profit_r2 = calculate_profit(predictions_valid_r2)
profit_r3 = calculate_profit(predictions_valid_r3)

In [19]:
print(f'O lucro da região 1 poderá ser de {profit_r1}')
print(f'O lucro da região 2 poderá ser de {profit_r2}')
print(f'O lucro da região 3 poderá ser de {profit_r3}')

O lucro da região 1 poderá ser de 39960488.77465132
O lucro da região 2 poderá ser de 24857120.51973544
O lucro da região 3 poderá ser de 33217543.962432623


In [20]:
positive_prob = (profit_r1 > 0).mean()
negative_prob = 1 - positive_prob
print(f"Probability of positive gross profit: {positive_prob:.2%}")
print(f"Probability of negative gross profit: {negative_prob:.2%}")

Probability of positive gross profit: 100.00%
Probability of negative gross profit: 0.00%


In [21]:
positive_prob = (profit_r2 > 0).mean()
negative_prob = 1 - positive_prob
print(f"Probability of positive gross profit: {positive_prob:.2%}")
print(f"Probability of negative gross profit: {negative_prob:.2%}")

Probability of positive gross profit: 100.00%
Probability of negative gross profit: 0.00%


In [22]:
positive_prob = (profit_r3 > 0).mean()
negative_prob = 1 - positive_prob
print(f"Probability of positive gross profit: {positive_prob:.2%}")
print(f"Probability of negative gross profit: {negative_prob:.2%}")

Probability of positive gross profit: 100.00%
Probability of negative gross profit: 0.00%


A região com maior predição de lucros é a região 1. Nenhuma das regiões possui probabilidade de prejuízos

## Calculo de riscos e lucro para cada região

In [23]:
# Função que irá calcular o lucro
def profit(target, probabilities):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index]
    revenue = selected.sum() * product_price
    return revenue - budget

In [24]:

state = np.random.RandomState(12345)
bootstrap_profit = []

for i in range(1000):

    subsample = pd.Series(predictions_valid_r1).sample(frac=1, replace=True, random_state=state)
    bootstrap_profit.append(profit(target_r1, subsample))

bootstrap_profit = pd.Series(bootstrap_profit)

mean_profit = bootstrap_profit.mean()
upper = bootstrap_profit.quantile(0.95)
lower = bootstrap_profit.quantile(0.05)
risk_of_loss = sum(bootstrap_profit < 0)* 100 / 1000


print("Region 1:")
print("Mean profit:", mean_profit)
print("95% confidence interval:", lower, upper)
print("Risk of loss:", risk_of_loss, "%")


Region 1:
Mean profit: 10352285964.992748
95% confidence interval: 10301401714.941599 10402517717.412786
Risk of loss: 0.0 %


In [25]:

state = np.random.RandomState(12345)
bootstrap_profit = []

for i in range(1000):

    subsample = pd.Series(predictions_valid_r2).sample(frac=1, replace=True, random_state=state)
    bootstrap_profit.append(profit(target_r2, subsample))

bootstrap_profit = pd.Series(bootstrap_profit)

mean_profit = bootstrap_profit.mean()
upper = bootstrap_profit.quantile(0.95)
lower = bootstrap_profit.quantile(0.05)
risk_of_loss = sum(bootstrap_profit < 0) * 100 / 1000

# Print the results
print("Region 2:")
print("Mean profit:", mean_profit)
print("95% confidence interval:", lower, upper)
print("Risk of loss:", risk_of_loss, "%")


Region 2:
Mean profit: 7613818418.48827
95% confidence interval: 7559502696.577531 7664332647.072469
Risk of loss: 0.0 %


In [26]:

state = np.random.RandomState(12345)
bootstrap_profit = []

for i in range(1000):

    subsample = pd.Series(predictions_valid_r3).sample(frac=1, replace=True, random_state=state)
    bootstrap_profit.append(profit(target_r3, subsample))

bootstrap_profit = pd.Series(bootstrap_profit)

mean_profit = bootstrap_profit.mean()
upper = bootstrap_profit.quantile(0.95)
lower = bootstrap_profit.quantile(0.05)
risk_of_loss = sum(bootstrap_profit < 0) / 1000
risk_of_loss_percent = risk_of_loss * 100

# Print the results
print("Region 3:")
print("Mean profit:", mean_profit)
print("95% confidence interval:", lower, upper)
print("Risk of loss:", risk_of_loss_percent, "%")


Region 3:
Mean profit: 10572860835.220417
95% confidence interval: 10518858704.6226 10626722391.549784
Risk of loss: 0.0 %


A região 3 apresentou maior média de lucro. Todas as regiões tiveram 0% de risco de perda.

## Conclusões

A região 3 parece ser o melhor local para construção de novos poços. É a região onde as predições mostram maiores volumes de reservas e, consequentemente, maiores receitas e lucros. Após aplicar a técnica de bootscrapping analysis a região 3 permaneceu como a região mais lucrativa com média de lucro bruto de 10572860835.220417 com 95% de probabilidade e risco de perdas de 0%. 